Для курсового проекта я выбрала тему - "Heart Failure"
Для этого взяла дата-сет "Heart Failurer"
https://www.kaggle.com/fedesoriano/heart-failure-prediction

Я хочу обучить модель, которая по симптомам будет прогнозировать развитие злокачественной опухоли

In [2]:
import pandas as pd
import numpy as np
import dill
import seaborn as sns
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


import itertools
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, \
        roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import matplotlib.pyplot as plt
import catboost as catb
%matplotlib inline

In [3]:
# input
DATASET_PATH = '../course_project/course_project_train.csv'

TEST_DATASET_PATH = 'heart.csv'




# output
TRAIN_X_FULL_PATH = 'hd_X_train.csv'
TRAIN_Y_FULL_PATH = "hd_y_train.csv"

TEST_X_FULL_PATH = "hd_X_test.csv"
TEST_Y_FULL_PATH = "hd_y_test.csv"

PREP_DATASET_PATH = '../course_project/project_train_data_prep.csv'

TRAIN_PART_PATH = '../course_project_train_part_b.csv'
TEST_PART_PATH = '../course_project_test_part.csv'

PIPELINE_FILE_PATH = "heart_decease_pipeline.dill"


In [4]:
df = pd.read_csv(TEST_DATASET_PATH)
df.head(3)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0


In [5]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


В качестве целевой переменной мы возьмем HeartDisease - . 

In [7]:
df['HeartDisease'].value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

### Обзор переменных

In [8]:
df['Sex'].value_counts()

M    725
F    193
Name: Sex, dtype: int64

In [9]:
sum(df['Sex'].isna())

0

In [10]:
df['ChestPainType'].value_counts()

ASY    496
NAP    203
ATA    173
TA      46
Name: ChestPainType, dtype: int64

In [11]:
sum(df['ChestPainType'].isna())

0

In [12]:
df['RestingECG'].value_counts()

Normal    552
LVH       188
ST        178
Name: RestingECG, dtype: int64

In [13]:
df['ExerciseAngina'].value_counts()

N    547
Y    371
Name: ExerciseAngina, dtype: int64

In [14]:
df['ST_Slope'].value_counts()

Flat    460
Up      395
Down     63
Name: ST_Slope, dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


На основе FeatureImportance пробной модели сократим количеством признаков, на которых обучается модель. уберем RestingECG, RestingBP, MaxHR, FastingBS

In [19]:
df.drop(axis=1, columns = ['RestingECG', 'RestingBP', 'MaxHR', 'FastingBS'], inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   Cholesterol     918 non-null    int64  
 4   ExerciseAngina  918 non-null    object 
 5   Oldpeak         918 non-null    float64
 6   ST_Slope        918 non-null    object 
 7   HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 57.5+ KB


In [25]:
cat_features = ['Sex','ChestPainType', 'ExerciseAngina', 'ST_Slope']

In [26]:
continuous_features = ['Age','Cholesterol','Oldpeak']

In [27]:
target = 'HeartDisease'

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [29]:
final_transformers = list()

for cat_col in cat_features:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_features:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

Объединим все это в единый пайплайн

In [30]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Теперь у нас есть пайплайн, который готовит признаки для моделирования.

Добавим модель

In [31]:
from sklearn.ensemble import RandomForestClassifier

pipeline_1 = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

Теперь разделим наш датасет на тестовую и тренировочную выборки

In [32]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=[target]), df[target],test_size=0.25,
                               shuffle=True, random_state=42)


In [33]:
#save test
X_test.to_csv(TEST_X_FULL_PATH, index=None)
y_test.to_csv(TEST_Y_FULL_PATH, index=None)
#save train
X_train.to_csv(TRAIN_X_FULL_PATH, index=None)
y_train.to_csv(TRAIN_Y_FULL_PATH, index=None)


### Обучение модели

In [34]:
#обучим наш пайплайн
pipeline_1.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('ExerciseAngina',
                                                 Pipeline(steps=[('selector',


In [35]:
from sklearn.model_selection import GridSearchCV

params_rf={'classifier__max_features':[0.3, 0.5, 0.7],
        'classifier__min_samples_leaf':[1, 2, 3],
        'classifier__max_depth':[3,5,7]
        }

In [40]:
%time
grid_rf = GridSearchCV(pipeline_1,
                    param_grid=params_rf,
                    cv=6,
                    refit=False)

search_rf = grid_rf.fit(X_train, y_train)
search_rf.best_params_

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 12.9 µs


{'classifier__max_depth': 7,
 'classifier__max_features': 0.3,
 'classifier__min_samples_leaf': 1}

In [41]:
pipeline_1 = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(max_depth = 7,\
                                          max_features=0.3, \
                                          random_state = 42,\
                                         min_samples_leaf =2)),
])

In [42]:
#обучим наш пайплайн
pipeline_1.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('ExerciseAngina',
                                                 Pipeline(steps=[('selector',


In [43]:

pipeline_2 = Pipeline([
    ('features',feats),
    ('classifier', catb.CatBoostClassifier(random_state = 42)),
])

In [45]:
params_cb={'classifier__iterations':[50, 100, 200],
        'classifier__max_depth':[3,5,7]
        }

In [46]:
%time
grid_cb = GridSearchCV(pipeline_2,
                    param_grid=params_cb,
                    cv=6,
                    refit=False)

search_cb = grid_cb.fit(X_train, y_train)


CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 21 µs
Learning rate set to 0.126681
0:	learn: 0.6396506	total: 71.1ms	remaining: 3.48s
1:	learn: 0.5999934	total: 73ms	remaining: 1.75s
2:	learn: 0.5612588	total: 74.5ms	remaining: 1.17s
3:	learn: 0.5313902	total: 75.9ms	remaining: 873ms
4:	learn: 0.5058978	total: 77.3ms	remaining: 696ms
5:	learn: 0.4860302	total: 78.6ms	remaining: 576ms
6:	learn: 0.4696593	total: 79.7ms	remaining: 490ms
7:	learn: 0.4533983	total: 81.1ms	remaining: 426ms
8:	learn: 0.4384939	total: 82.2ms	remaining: 374ms
9:	learn: 0.4269121	total: 83.3ms	remaining: 333ms
10:	learn: 0.4148465	total: 84.4ms	remaining: 299ms
11:	learn: 0.4075709	total: 85.4ms	remaining: 270ms
12:	learn: 0.3992330	total: 86.4ms	remaining: 246ms
13:	learn: 0.3926396	total: 87.2ms	remaining: 224ms
14:	learn: 0.3832423	total: 88ms	remaining: 205ms
15:	learn: 0.3768977	total: 89ms	remaining: 189ms
16:	learn: 0.3713167	total: 89.8ms	remaining: 174ms
17:	learn: 0.3676245	total: 90.6ms	rema

41:	learn: 0.3047014	total: 53.9ms	remaining: 10.3ms
42:	learn: 0.3039056	total: 58.1ms	remaining: 9.45ms
43:	learn: 0.3019185	total: 59.4ms	remaining: 8.1ms
44:	learn: 0.3005197	total: 60.3ms	remaining: 6.7ms
45:	learn: 0.3001006	total: 61.3ms	remaining: 5.33ms
46:	learn: 0.2992207	total: 63.4ms	remaining: 4.05ms
47:	learn: 0.2984702	total: 64.7ms	remaining: 2.7ms
48:	learn: 0.2975599	total: 66ms	remaining: 1.35ms
49:	learn: 0.2971032	total: 67.3ms	remaining: 0us
Learning rate set to 0.126775
0:	learn: 0.6458743	total: 1.24ms	remaining: 60.9ms
1:	learn: 0.6035110	total: 2.27ms	remaining: 54.6ms
2:	learn: 0.5653221	total: 4.06ms	remaining: 63.6ms
3:	learn: 0.5359757	total: 5.03ms	remaining: 57.8ms
4:	learn: 0.5086329	total: 6.07ms	remaining: 54.6ms
5:	learn: 0.4845370	total: 7.14ms	remaining: 52.4ms
6:	learn: 0.4680660	total: 8.17ms	remaining: 50.2ms
7:	learn: 0.4510282	total: 9.44ms	remaining: 49.6ms
8:	learn: 0.4359598	total: 10.6ms	remaining: 48.1ms
9:	learn: 0.4214754	total: 11.7ms

30:	learn: 0.3101292	total: 44.1ms	remaining: 27ms
31:	learn: 0.3083142	total: 47.9ms	remaining: 26.9ms
32:	learn: 0.3047242	total: 49.4ms	remaining: 25.5ms
33:	learn: 0.3023891	total: 51.1ms	remaining: 24ms
34:	learn: 0.3001401	total: 52.6ms	remaining: 22.6ms
35:	learn: 0.2991747	total: 55.4ms	remaining: 21.5ms
36:	learn: 0.2974951	total: 56.9ms	remaining: 20ms
37:	learn: 0.2962119	total: 58.1ms	remaining: 18.3ms
38:	learn: 0.2939339	total: 59.3ms	remaining: 16.7ms
39:	learn: 0.2921235	total: 60.8ms	remaining: 15.2ms
40:	learn: 0.2895868	total: 62.4ms	remaining: 13.7ms
41:	learn: 0.2885829	total: 64ms	remaining: 12.2ms
42:	learn: 0.2866607	total: 65.5ms	remaining: 10.7ms
43:	learn: 0.2854863	total: 67ms	remaining: 9.14ms
44:	learn: 0.2839946	total: 68.1ms	remaining: 7.57ms
45:	learn: 0.2827669	total: 69.3ms	remaining: 6.03ms
46:	learn: 0.2821176	total: 70.5ms	remaining: 4.5ms
47:	learn: 0.2808430	total: 71.7ms	remaining: 2.99ms
48:	learn: 0.2792639	total: 72.8ms	remaining: 1.49ms
49:	

48:	learn: 0.2791026	total: 44.9ms	remaining: 916us
49:	learn: 0.2776111	total: 46.3ms	remaining: 0us
Learning rate set to 0.126775
0:	learn: 0.6398578	total: 1.39ms	remaining: 68ms
1:	learn: 0.5912204	total: 2.67ms	remaining: 64ms
2:	learn: 0.5527816	total: 3.89ms	remaining: 60.9ms
3:	learn: 0.5221241	total: 5.08ms	remaining: 58.5ms
4:	learn: 0.4951865	total: 5.91ms	remaining: 53.2ms
5:	learn: 0.4731005	total: 7.37ms	remaining: 54ms
6:	learn: 0.4518613	total: 8.65ms	remaining: 53.1ms
7:	learn: 0.4345081	total: 9.8ms	remaining: 51.4ms
8:	learn: 0.4168213	total: 11.1ms	remaining: 50.4ms
9:	learn: 0.4047166	total: 12.2ms	remaining: 49ms
10:	learn: 0.3931604	total: 13.4ms	remaining: 47.4ms
11:	learn: 0.3810922	total: 14.5ms	remaining: 46.1ms
12:	learn: 0.3719344	total: 15.7ms	remaining: 44.7ms
13:	learn: 0.3646180	total: 17ms	remaining: 43.7ms
14:	learn: 0.3588514	total: 18ms	remaining: 41.9ms
15:	learn: 0.3519428	total: 19.1ms	remaining: 40.6ms
16:	learn: 0.3476095	total: 20.2ms	remainin

6:	learn: 0.4645493	total: 15.7ms	remaining: 96.4ms
7:	learn: 0.4470211	total: 18.7ms	remaining: 98.1ms
8:	learn: 0.4304873	total: 21.3ms	remaining: 97.2ms
9:	learn: 0.4165669	total: 23.8ms	remaining: 95.1ms
10:	learn: 0.4036243	total: 26.3ms	remaining: 93.2ms
11:	learn: 0.3934784	total: 28.9ms	remaining: 91.4ms
12:	learn: 0.3841090	total: 31.2ms	remaining: 88.9ms
13:	learn: 0.3752342	total: 33.4ms	remaining: 85.9ms
14:	learn: 0.3667015	total: 36ms	remaining: 84.1ms
15:	learn: 0.3592843	total: 38.7ms	remaining: 82.2ms
16:	learn: 0.3520724	total: 41.2ms	remaining: 80.1ms
17:	learn: 0.3456759	total: 43.5ms	remaining: 77.4ms
18:	learn: 0.3379869	total: 45.6ms	remaining: 74.4ms
19:	learn: 0.3330162	total: 47.7ms	remaining: 71.6ms
20:	learn: 0.3274240	total: 49.7ms	remaining: 68.6ms
21:	learn: 0.3239125	total: 51.6ms	remaining: 65.7ms
22:	learn: 0.3201437	total: 53.7ms	remaining: 63ms
23:	learn: 0.3149986	total: 56.2ms	remaining: 60.8ms
24:	learn: 0.3105018	total: 58.5ms	remaining: 58.5ms
2

...

144:	learn: 0.1648167	total: 388ms	remaining: 147ms
145:	learn: 0.1638857	total: 392ms	remaining: 145ms
146:	learn: 0.1638624	total: 393ms	remaining: 142ms
147:	learn: 0.1634775	total: 396ms	remaining: 139ms
148:	learn: 0.1630521	total: 399ms	remaining: 137ms
149:	learn: 0.1624428	total: 402ms	remaining: 134ms
150:	learn: 0.1615590	total: 405ms	remaining: 131ms
151:	learn: 0.1607357	total: 408ms	remaining: 129ms
152:	learn: 0.1603318	total: 410ms	remaining: 126ms
153:	learn: 0.1598495	total: 412ms	remaining: 123ms
154:	learn: 0.1593209	total: 415ms	remaining: 120ms
155:	learn: 0.1587563	total: 418ms	remaining: 118ms
156:	learn: 0.1581993	total: 421ms	remaining: 115ms
157:	learn: 0.1576941	total: 424ms	remaining: 113ms
158:	learn: 0.1576123	total: 425ms	remaining: 110ms
159:	learn: 0.1569715	total: 429ms	remaining: 107ms
160:	learn: 0.1564875	total: 432ms	remaining: 105ms
161:	learn: 0.1556065	total: 434ms	remaining: 102ms
162:	learn: 0.1550716	total: 437ms	remaining: 99.2ms
163:	learn:

151:	learn: 0.1548329	total: 368ms	remaining: 116ms
152:	learn: 0.1542515	total: 373ms	remaining: 114ms
153:	learn: 0.1536725	total: 376ms	remaining: 112ms
154:	learn: 0.1536063	total: 378ms	remaining: 110ms
155:	learn: 0.1531642	total: 381ms	remaining: 108ms
156:	learn: 0.1522723	total: 385ms	remaining: 105ms
157:	learn: 0.1508803	total: 388ms	remaining: 103ms
158:	learn: 0.1500732	total: 390ms	remaining: 101ms
159:	learn: 0.1496495	total: 392ms	remaining: 98.1ms
160:	learn: 0.1493533	total: 396ms	remaining: 95.8ms
161:	learn: 0.1490336	total: 399ms	remaining: 93.5ms
162:	learn: 0.1485104	total: 402ms	remaining: 91.3ms
163:	learn: 0.1479365	total: 405ms	remaining: 88.9ms
164:	learn: 0.1475069	total: 407ms	remaining: 86.4ms
165:	learn: 0.1469136	total: 410ms	remaining: 84ms
166:	learn: 0.1462962	total: 412ms	remaining: 81.5ms
167:	learn: 0.1455887	total: 415ms	remaining: 79ms
168:	learn: 0.1448914	total: 417ms	remaining: 76.5ms
169:	learn: 0.1445781	total: 420ms	remaining: 74.1ms
170:	

159:	learn: 0.1481510	total: 370ms	remaining: 92.6ms
160:	learn: 0.1469315	total: 375ms	remaining: 90.8ms
161:	learn: 0.1469187	total: 376ms	remaining: 88.3ms
162:	learn: 0.1463227	total: 380ms	remaining: 86.2ms
163:	learn: 0.1456156	total: 383ms	remaining: 84.1ms
164:	learn: 0.1448928	total: 387ms	remaining: 82ms
165:	learn: 0.1445002	total: 390ms	remaining: 79.8ms
166:	learn: 0.1439837	total: 394ms	remaining: 77.8ms
167:	learn: 0.1432487	total: 397ms	remaining: 75.6ms
168:	learn: 0.1425987	total: 400ms	remaining: 73.4ms
169:	learn: 0.1425481	total: 401ms	remaining: 70.8ms
170:	learn: 0.1418642	total: 404ms	remaining: 68.5ms
171:	learn: 0.1406755	total: 407ms	remaining: 66.2ms
172:	learn: 0.1398074	total: 409ms	remaining: 63.8ms
173:	learn: 0.1392467	total: 412ms	remaining: 61.6ms
174:	learn: 0.1386684	total: 415ms	remaining: 59.3ms
175:	learn: 0.1382248	total: 418ms	remaining: 57ms
176:	learn: 0.1376682	total: 421ms	remaining: 54.7ms
177:	learn: 0.1371079	total: 424ms	remaining: 52.4

155:	learn: 0.1597028	total: 372ms	remaining: 105ms
156:	learn: 0.1594157	total: 375ms	remaining: 103ms
157:	learn: 0.1589410	total: 377ms	remaining: 100ms
158:	learn: 0.1588568	total: 379ms	remaining: 97.7ms
159:	learn: 0.1584826	total: 381ms	remaining: 95.3ms
160:	learn: 0.1576931	total: 383ms	remaining: 92.9ms
161:	learn: 0.1569530	total: 385ms	remaining: 90.4ms
162:	learn: 0.1566272	total: 388ms	remaining: 88ms
163:	learn: 0.1562640	total: 391ms	remaining: 85.8ms
164:	learn: 0.1556012	total: 393ms	remaining: 83.4ms
165:	learn: 0.1551024	total: 396ms	remaining: 81.2ms
166:	learn: 0.1544855	total: 400ms	remaining: 78.9ms
167:	learn: 0.1537959	total: 403ms	remaining: 76.8ms
168:	learn: 0.1530436	total: 406ms	remaining: 74.4ms
169:	learn: 0.1522834	total: 409ms	remaining: 72.2ms
170:	learn: 0.1515270	total: 412ms	remaining: 69.8ms
171:	learn: 0.1510176	total: 414ms	remaining: 67.5ms
172:	learn: 0.1499899	total: 417ms	remaining: 65.1ms
173:	learn: 0.1497356	total: 419ms	remaining: 62.7m

145:	learn: 0.1460972	total: 371ms	remaining: 137ms
146:	learn: 0.1459246	total: 375ms	remaining: 135ms
147:	learn: 0.1457490	total: 378ms	remaining: 133ms
148:	learn: 0.1450807	total: 381ms	remaining: 130ms
149:	learn: 0.1445622	total: 384ms	remaining: 128ms
150:	learn: 0.1439707	total: 388ms	remaining: 126ms
151:	learn: 0.1437993	total: 391ms	remaining: 123ms
152:	learn: 0.1433448	total: 393ms	remaining: 121ms
153:	learn: 0.1426868	total: 396ms	remaining: 118ms
154:	learn: 0.1418240	total: 399ms	remaining: 116ms
155:	learn: 0.1407839	total: 402ms	remaining: 113ms
156:	learn: 0.1401769	total: 405ms	remaining: 111ms
157:	learn: 0.1392201	total: 409ms	remaining: 109ms
158:	learn: 0.1388834	total: 411ms	remaining: 106ms
159:	learn: 0.1383879	total: 414ms	remaining: 104ms
160:	learn: 0.1378295	total: 417ms	remaining: 101ms
161:	learn: 0.1369627	total: 419ms	remaining: 98.3ms
162:	learn: 0.1369456	total: 420ms	remaining: 95.4ms
163:	learn: 0.1362099	total: 423ms	remaining: 92.9ms
164:	lear

In [47]:
search_cb.best_params_

{'classifier__iterations': 200, 'classifier__max_depth': 3}

In [48]:

pipeline_2 = Pipeline([
    ('features',feats),
    ('classifier', catb.CatBoostClassifier(iterations = 200,\
                                           max_depth=3,
                                           random_state = 42)),
])


In [49]:
pipeline_2.fit(X_train, y_train)

Learning rate set to 0.038418
0:	learn: 0.6648744	total: 1.97ms	remaining: 391ms
1:	learn: 0.6456676	total: 3.84ms	remaining: 380ms
2:	learn: 0.6242099	total: 5.35ms	remaining: 351ms
3:	learn: 0.6031473	total: 7.01ms	remaining: 343ms
4:	learn: 0.5828988	total: 8.53ms	remaining: 333ms
5:	learn: 0.5629649	total: 10.3ms	remaining: 331ms
6:	learn: 0.5437890	total: 11.5ms	remaining: 317ms
7:	learn: 0.5327356	total: 12.8ms	remaining: 307ms
8:	learn: 0.5207788	total: 14.1ms	remaining: 300ms
9:	learn: 0.5051044	total: 15.5ms	remaining: 294ms
10:	learn: 0.4957560	total: 16.7ms	remaining: 287ms
11:	learn: 0.4860590	total: 18ms	remaining: 281ms
12:	learn: 0.4752287	total: 19.2ms	remaining: 277ms
13:	learn: 0.4649557	total: 20.6ms	remaining: 274ms
14:	learn: 0.4578381	total: 21.9ms	remaining: 270ms
15:	learn: 0.4501429	total: 23.2ms	remaining: 267ms
16:	learn: 0.4403168	total: 24.6ms	remaining: 265ms
17:	learn: 0.4353034	total: 25.9ms	remaining: 262ms
18:	learn: 0.4276963	total: 27.2ms	remaining: 

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('ExerciseAngina',
                                                 Pipeline(steps=[('selector',


In [50]:
predictions1 = pipeline_1.predict_proba(X_test)
predictions2 = pipeline_2.predict_proba(X_test)

In [51]:
def get_model_scores(y_test, y_score):
    b=1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test, y_score)
    return precision[ix], recall[ix], fscore[ix], roc_auc, thresholds[ix]

In [52]:
pr1, rec1, f1, roc1, thresh1 =  get_model_scores(y_test, predictions1[:, 1][:])
pr1, rec1, f1, roc1, thresh1 

(0.8854961832061069,
 0.8787878787878788,
 0.8821292775665399,
 0.9221938775510203,
 0.4288438592790484)

In [53]:
pr2, rec2, f2, roc2, thresh2 =  get_model_scores(y_test, predictions2[:, 1][:])
pr2, rec2, f2, roc2, thresh2 

(0.8776978417266187,
 0.9242424242424242,
 0.9003690036900369,
 0.9355674087816945,
 0.33601411138028925)

Выбираем pipeline_2, потому что у него метрики более высокие

In [54]:
def show_feature_importances(feature_names, feature_importances, get_top=None):
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
    feature_importances = feature_importances.sort_values('importance', ascending=False)
       
    plt.figure(figsize = (20, len(feature_importances) * 0.355))
    
    sns.barplot(feature_importances['importance'], feature_importances['feature'])
    
    plt.xlabel('Importance')
    plt.title('Importance of features')
    plt.show()
    
    if get_top is not None:
        return feature_importances['feature'][:get_top].tolist()

important_features_top = show_feature_importances(X_train.columns, model_catb.feature_importances_, get_top=23)

На основе FeatureImportance сократим количеством признаков, на которых обучается модель.
уберем RestingECG, RestingBP, MaxHR, FastingBS

In [56]:
model_dict = {
    'model':pipeline_2,\
    'version':'0.0.1'
    
}

In [57]:
with open(PIPELINE_FILE_PATH, "wb") as f:
    dill.dump(model_dict, f)